<a href="https://colab.research.google.com/github/marco-siino/llm-network-testbed/blob/main/knowledge-extraction/Q2/Codestral_LogKnowledgeExtraction_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [1]:
import os
import re
import random
import time
import requests
from google.colab import files
import shutil
!pip install rouge-score

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

!pip install mistralai
from mistralai import Mistral

#api_key = os.environ['7yOu0lH5XcZC1U2ZUI2uv0ghBDduhHp5']

client = Mistral(api_key='7yOu0lH5XcZC1U2ZUI2uv0ghBDduhHp5')

model = "codestral-mamba-latest"

message = [
    {
        "role": "user",
        "content": "Write a function for fibonacci"
    }
]

chat_response = client.chat.complete(
    model=model,
    messages=message
)
print(chat_response.choices[0].message.content)

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=20cab253f0ed9f5abb5b2f7f76893e79b2837ee54b9749569ad69414416cfc68
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 11.3 MB/s eta 0:00:00
Sure, here's a simple Python function for generating Fibonacci numbers:

```python
def fibonacci(n):
    if n <= 0:
        return "Input should be a positive integer"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        fib_seq = [0, 1]
        while len(fib_seq) < n:
            fib_seq.append(fib_seq[-1] + fib_seq[-2])
        return fib_seq[-1]
```

You can use this function to get the nth Fibonacci number by calling `fibonacci(n)`. For example, `fibonacci(10)` will return 34, which is the 10th Fibonacci number.


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


# Download and organize the dataset.

In [4]:
url = "https://github.com/marco-siino/llm-network-testbed/raw/refs/heads/main/knowledge-extraction/Q2/prompt_Q2-2024-06-13.zip"

output_file = "prompt_Q2-2024-06-13.zip"

os.system(f"wget -O {output_file} {url}")

0

In [5]:
!unzip prompt_Q2-2024-06-13.zip

Archive:  prompt_Q2-2024-06-13.zip
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1046.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=992.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=966.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=982.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1084.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1012.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1086.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1050.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1076.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1070.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1062.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1042.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1010.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1058.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1088.txt  
  inflating: prompt_Q2/prompt_n_hours=

In [6]:
folder_path="prompt_Q2"
# List all files in the folder
files = os.listdir(folder_path)

# Sort the files alphabetically
files.sort()

# Function to pre-process the few shots.

In [7]:
# The following function add the Mistral Tags within the file content.
def preprocess_sample(file_contents):

                log_sample ="Given the following LOG, which dev_eui have retransmitted the same frame at least once?\n\nLOG:\n"

                parts = file_contents.split('QUESTION:')
                log_sample += parts[0]
                log_sample += "ANSWER:"

                parts = file_contents.split('ANSWER:')
                answer = parts[1]

                question = log_sample
                answer =  answer

                return question,answer


In [8]:
file_contents = """
time: 2019-05-20 06:17:19.778000, freq: 868.5, chan: 23, lsnr: 10.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4092
time: 2019-05-20 06:26:27.954000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:26:34.149000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:38:22.054000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3920
time: 2019-05-20 06:40:28.948000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:40:35.076000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:47:13.875000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4050
time: 2019-05-20 06:49:00.357000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 148
time: 2019-05-20 07:12:18.815000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3850
time: 2019-05-20 07:17:18.782000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4093
time: 2019-05-20 07:19:13.464000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4106
time: 2019-05-20 07:26:33.254000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3849
time: 2019-05-20 07:26:39.384000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3849
time: 2019-05-20 07:38:21.808000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3921
time: 2019-05-20 07:40:28.025000, freq: 868.5, chan: 23, lsnr: -6.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4100
time: 2019-05-20 07:40:34.159000, freq: 868.5, chan: 23, lsnr: 1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4100
time: 2019-05-20 07:47:12.982000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4051
time: 2019-05-20 07:48:59.461000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 149
time: 2019-05-20 08:12:18.419000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3851


QUESTION: Which dev_eui have retransmitted the same frame at least once in 2 hours?

ANSWER:
 dev_eui: 121
 dev_eui: 124

"""

log_sample, answer = preprocess_sample(file_contents)

print(log_sample, answer)

Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:

time: 2019-05-20 06:17:19.778000, freq: 868.5, chan: 23, lsnr: 10.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4092
time: 2019-05-20 06:26:27.954000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:26:34.149000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 3848
time: 2019-05-20 06:38:22.054000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3920
time: 2019-05-20 06:40:28.948000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:40:35.076000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4099
time: 2019-05-20 06:47:13.8

# Create the few-shots samples.

In [9]:
def create_few_shots_set(folder_path,nr_samples):

    # Counter to keep track of the number of files read
    count = 0

    few_shots_set = ""

    # Get a list of all files in the directory
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Shuffle the list of files. Seed set - equal to 4 - for reproducibility.
    random.Random(4).shuffle(files)

    global test_set
    test_set = files

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                question, answer = preprocess_sample(file_contents)

                few_shots_set += question
                few_shots_set += answer

                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == nr_samples:
                    break
        test_set.remove(filename)

    return few_shots_set

In [10]:
# Specify the number of files to read
n = 3  # Example: Read the first 5 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)

prompt_n_hours=2_skip_hours=760.txt
prompt_n_hours=2_skip_hours=134.txt
prompt_n_hours=2_skip_hours=286.txt
Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-06-20 16:14:09.324000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4890
time: 2019-06-20 16:15:46.183000, freq: 868.3, chan: 22, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4634
time: 2019-06-20 16:19:16.057000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 182
time: 2019-06-20 16:29:36.219000, freq: 868.5, chan: 23, lsnr: 12.0, rssi: -102.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4885
time: 2019-06-20 16:33:20.585000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4706
time: 2019-06-20 16:36:24.505000, freq: 868.5, chan: 23, lsnr: 15

# Chat with the model to get the results.

In [11]:
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

# Percorso dove vuoi salvare il file ZIP su Google Drive
path_google_drive = '/content/drive/MyDrive/codestral_results_q2.zip'

Mounted at /content/drive


In [12]:
# Define the directory path containing the results.
result_folder_name = 'codestral_results'
result_directory_path = os.path.join(os.getcwd(), result_folder_name)
dataset_path = 'prompt_Q2'

# Iterate through all the test_files
for filename in test_set:
        print(filename)
        if(os.path.isfile(result_directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Create the directory if it doesn't exist
            if not os.path.exists(result_directory_path):
                os.makedirs(result_directory_path)

            # Open the file and read its contents
            with open(os.path.join(dataset_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                chat = ""
                # Specify the number of files to read
                #n = 5  # Example: Read the first 5 files
                # Call the function to read text files in the folder and concatenate their contents
                #few_shots_samples = create_few_shots_set(folder_path, n)

                chat = few_shots_samples
                question,anwer = preprocess_sample(file_contents)

                chat+= question

                prompt = chat
                print("****.  ECCCO IL PROMPT !!!! **** ")
                print(prompt)
                print("\n\n\n\n")

                message = [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]

                success = False
                while not success:
                    try:
                        chat_response = client.chat.complete(
                            model=model,
                            messages=message
                        )
                        success = True  # Se non viene sollevata alcuna eccezione, imposta success su True
                    except Exception as e:
                        print(f"Errore: {e}. Riprovo tra 20 secondi...")
                        time.sleep(20)  # Attendi 20 secondi prima di ritentare


                result = chat_response.choices[0].message.content
                print(result+"\n\n")

                # Define the file path
                file_path = os.path.join(result_directory_path, filename)

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

                # Zip the result folder and download it on Drive.
                # Comprime la cartella in un file zip e salvalo su Google Drive
                shutil.make_archive(path_google_drive.replace('.zip', ''), 'zip', result_folder_name)

                # Ora puoi andare su Google Drive per scaricare il file ZIP
                print("File compresso e salvato su Google Drive.")

prompt_n_hours=2_skip_hours=836.txt
Already done!


prompt_n_hours=2_skip_hours=198.txt
Already done!


prompt_n_hours=2_skip_hours=286.txt
Already done!


prompt_n_hours=2_skip_hours=796.txt
Already done!


prompt_n_hours=2_skip_hours=602.txt
Already done!


prompt_n_hours=2_skip_hours=812.txt
Already done!


prompt_n_hours=2_skip_hours=910.txt
Already done!


prompt_n_hours=2_skip_hours=608.txt
Already done!


prompt_n_hours=2_skip_hours=54.txt
Already done!


prompt_n_hours=2_skip_hours=872.txt
Already done!


prompt_n_hours=2_skip_hours=84.txt
Already done!


prompt_n_hours=2_skip_hours=666.txt
Already done!


prompt_n_hours=2_skip_hours=870.txt
Already done!


prompt_n_hours=2_skip_hours=522.txt
Already done!


prompt_n_hours=2_skip_hours=738.txt
Already done!


prompt_n_hours=2_skip_hours=620.txt
Already done!


prompt_n_hours=2_skip_hours=210.txt
Already done!


prompt_n_hours=2_skip_hours=912.txt
Already done!


prompt_n_hours=2_skip_hours=688.txt
****.  ECCCO IL PROMPT !!!! **

In [13]:
# Count and print the number of file within the codestral_results folder
file_count = sum(1 for _ in os.listdir(result_folder_name) if os.path.isfile(os.path.join(result_folder_name, _)))
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 546


In [ ]:
# Count and print the number of file within the codestral_results folder
file_count = sum(1 for _ in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, _)))
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 546


# Given the same few-shot samples ask to Codestral to generate the Python code to perform the task.

In [14]:
prompt ="""
Given the following string of a LOG, write a Python function to OUTPUT (as in the example below) which dev_eui has retransmitted the same frame at least once.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889


OUTPUT:
dev_eui: 121
dev_eui: 124


"""

In [15]:
message = [
    {
        "role": "user",
        "content": prompt
    }
]

chat_response = client.chat.complete(
    model=model,
    messages=message
)

result = chat_response.choices[0].message.content
print(result+"\n\n")

Here is a Python function that output the dev_eui of devices that have retransmitted the same frame at least once:

```python
def find_retransmitted_frames(log):
    lines = log.split('\n')
    lines = [line.split(', ') for line in lines if line]  # Split each line into a list of key-value pairs

    retransmitted_devices = set()  # To store the dev_eui of devices that have retransmitted frames
    frame_counts = {}  # To store the count of each frame

    for line in lines:
        if line:
            frame_info = {item.split(': ')[0]: item.split(': ')[1] for item in line}  # Convert each line into a dictionary
            frame = (frame_info['freq'], frame_info['SF'], frame_info['BW'], frame_info['codr'])  # Create a tuple to represent the frame

            if frame in frame_counts:
                frame_counts[frame] += 1
            else:
                frame_counts[frame] = 1

            if frame_counts[frame] > 1:
                retransmitted_devices.add(frame_info['dev_eui'

In [22]:
# Let's write a function to use this code.
# To be adherent as much as possible to the original question we try to not change almost anything.
def find_retransmitted_frames(log):
    lines = log.split('\n')
    lines = [line.split(', ') for line in lines if line]  # Split each line into a list of key-value pairs

    retransmitted_devices = set()  # To store the dev_eui of devices that have retransmitted frames
    frame_counts = {}  # To store the count of each frame

    for line in lines:
        if line:
            frame_info = {item.split(': ')[0]: item.split(': ')[1] for item in line}  # Convert each line into a dictionary
            try:
              frame = (frame_info['freq'], frame_info['SF'], frame_info['BW'], frame_info['codr'])  # Create a tuple to represent the frame
            except:
              continue
            if frame in frame_counts:
                frame_counts[frame] += 1
            else:
                frame_counts[frame] = 1

            if frame_counts[frame] > 1:
                retransmitted_devices.add(frame_info['dev_eui'])

    for device in retransmitted_devices:
        print('dev_eui:', device)
    output = ""
    for dev_eui in retransmitted_devices:
        output += f"dev_eui: {dev_eui}\n"

    return output

# Test the function with the provided log
log = '''time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889'''

find_retransmitted_frames(log)

dev_eui: 126
dev_eui: 128
dev_eui: 121
dev_eui: 124
dev_eui: 122
dev_eui: 123
dev_eui: 127
dev_eui: 120


'dev_eui: 126\ndev_eui: 128\ndev_eui: 121\ndev_eui: 124\ndev_eui: 122\ndev_eui: 123\ndev_eui: 127\ndev_eui: 120\n'

## Let's run the code.

In [23]:
# Define the directory path containing the results.
#folder_name = 'codestral_source_code_results/'

result_folder_name = 'codestral_source_code_results'
result_directory_path = os.path.join(os.getcwd(), result_folder_name)
dataset_path = 'prompt_Q2'

#directory_path = os.path.join(os.getcwd(), folder_name)
# Create the directory if it doesn't exist
if not os.path.exists(result_folder_name):
  os.makedirs(result_folder_name)

# Iterate through all the test_files
for filename in test_set:
        file_path = result_directory_path+"/"+filename
        #print(filename)
        #print(file_path)
        if(os.path.isfile(result_directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Open the file and read its contents
            with open(os.path.join(dataset_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                # Remove the ANSWER in the original log file.
                file_contents = file_contents.split("ANSWER", 1)
                print(file_contents[0])

                result = find_retransmitted_frames(file_contents[0])

                print(str(result)+"\n\n")

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

time: 2019-06-23 20:13:40.584000, freq: 868.1, chan: 21, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4969
time: 2019-06-23 20:14:35.363000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4713
time: 2019-06-23 20:14:41.492000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4713
time: 2019-06-23 20:18:35.316000, freq: 868.5, chan: 23, lsnr: 4.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 261
time: 2019-06-23 20:32:51.107000, freq: 868.1, chan: 21, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4785
time: 2019-06-23 20:35:18.818000, freq: 868.5, chan: 23, lsnr: 14.0, rssi: -81.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4915
time: 2019-06-23 20:37:31.094000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -89.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 12

# Here the function to run the syntactic evaluation at file-level and at global-level.

In [24]:
def evaluate_results (dataset_path,result_path):
  samples_nr = 0
  global_bleu = 0
  global_rouge1_fmeasure = 0
  global_rougeL_fmeasure = 0
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      print(text2)
      print("\n")
      print(text1)

      #dataset_corpus += text2
      #results_corpus += text1

      # Tokenize the texts
      reference = text1.split()
      candidate = text2.split()

      # Compute BLEU score
      smoothing_function = SmoothingFunction().method1
      bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

      # Compute ROUGE score
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      rouge_scores = scorer.score(text1, text2)

      # Print the results
      print(f"BLEU score: {bleu_score}")
      print(f"ROUGE-1: {rouge_scores['rouge1']}")
      print(f"ROUGE-L: {rouge_scores['rougeL']}")

      print("\n\n")


      global_bleu += bleu_score
      global_rouge1_fmeasure += rouge_scores['rouge1'].fmeasure
      global_rougeL_fmeasure += rouge_scores['rougeL'].fmeasure

      samples_nr += 1
  # Now let's print the results on the whole corpus.
  print("\n\nTHE RESULT ON THE TWO CORPORA ARE:\n")
  # Tokenize the texts
  #reference = results_corpus.split()
  #candidate = dataset_corpus.split()

  global_bleu = global_bleu/samples_nr
  global_rouge1_fmeasure = global_rouge1_fmeasure/samples_nr
  global_rougeL_fmeasure = global_rougeL_fmeasure/samples_nr

  # Print the results
  print(f"BLEU score: {global_bleu}")
  print(f"ROUGE-1: {global_rouge1_fmeasure}")
  print(f"ROUGE-L: {global_rougeL_fmeasure}")

## Let's evaluate the results of Codestral.

In [25]:
dataset_path = "prompt_Q2/"
result_path = "codestral_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.



Sample:  prompt_n_hours=2_skip_hours=422.txt
dev_eui: 126 
 dev_eui: 128


dev_eui: 122
dev_eui: 124
dev_eui: 121
dev_eui: 123
dev_eui: 127
dev_eui: 120
dev_eui: 125
dev_eui: 128
dev_eui: 126
BLEU score: 0.00725586175641505
ROUGE-1: Score(precision=1.0, recall=0.2222222222222222, fmeasure=0.3636363636363636)
ROUGE-L: Score(precision=0.8333333333333334, recall=0.18518518518518517, fmeasure=0.303030303030303)



Sample:  prompt_n_hours=2_skip_hours=86.txt
dev_eui: 124


dev_eui: 123
dev_eui: 124
dev_eui: 128
BLEU score: 0.04279677428117006
ROUGE-1: Score(precision=1.0, recall=0.3333333333333333, fmeasure=0.5)
ROUGE-L: Score(precision=1.0, recall=0.3333333333333333, fmeasure=0.5)



Sample:  prompt_n_hours=2_skip_hours=4.txt
dev_eui: 121


dev_eui: 122
dev_eui: 124
dev_eui: 128
dev_eui: 126

Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-05-17 15:19:19.735000, freq: 868.5, chan:

## Let's evaluate the results of the source code generated by Codestral.

In [26]:
dataset_path = "prompt_Q2/"
result_path = "codestral_source_code_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
dev_eui: 127
dev_eui: 120

BLEU score: 0.038815599336794765
ROUGE-1: Score(precision=1.0, recall=0.2857142857142857, fmeasure=0.4444444444444445)
ROUGE-L: Score(precision=0.8333333333333334, recall=0.23809523809523808, fmeasure=0.37037037037037035)



Sample:  prompt_n_hours=2_skip_hours=56.txt
dev_eui: 124 
 dev_eui: 128


dev_eui: 128
dev_eui: 121
dev_eui: 124
dev_eui: 123
dev_eui: 127
dev_eui: 120

BLEU score: 0.06399610426154731
ROUGE-1: Score(precision=1.0, recall=0.3333333333333333, fmeasure=0.5)
ROUGE-L: Score(precision=0.8333333333333334, recall=0.2777777777777778, fmeasure=0.4166666666666667)



Sample:  prompt_n_hours=2_skip_hours=1008.txt
no retransmissions


dev_eui: 126
dev_eui: 125
dev_eui: 128
dev_eui: 121
dev_eui: 124
dev_eui: 122
dev_eui: 123
dev_eui: 127
dev_eui: 129
dev_eui: 120

BLEU score: 0
ROUGE-1: Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE-L: Score(precision=0.0, recall=0.0, fmeasure=0.0)



Sample:  p

#Let's evaluate the correctness of the responses in terms of Precision and Recall.

In [27]:
def calculate_precision_recall (dataset_path,result_path):
  total_true_positive = 0
  total_false_positive = 0
  total_false_negative = 0
  # Iterate through all the test_files
  loop=0
  for filename in test_set:
      #loop+=1
      #if loop==20:
        #break
      # Specify the paths to the files
      result_file_path = result_path+filename
      dataset_file_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(result_file_path, 'r', encoding='utf-8') as file1:
          result_file_content = file1.read()
      except:
        continue
      try:
        with open(dataset_file_path, 'r', encoding='utf-8') as file2:
          test_file_content = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = test_file_content.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          test_file_content = test_file_content[answer_start_index + len("ANSWER:"):].strip()

      print("\n\n * * * Sample: ", filename)

      print(test_file_content)
      print("\n")
      print(result_file_content)

      # Create a list for both text files. Key is the dev_eui and value is the number of frames.

      # For each line in text1 create a pair key value. key is the number of dev_eui, and value is the number of frames.
      results_dict = {}
      target_dict = {}

      # Split the data into lines
      lines = test_file_content.split('\n')

      #print(lines)

      for line in lines:
        # Split the line to extract dev_eui and frames
        parts = line.split()
        try:
          dev_eui = parts[1]

        except:
          continue

        # Add to dictionary
        if dev_eui == 'retransmissions':
            continue
        try:
          target_dict[dev_eui] = dev_eui
        except:
          target_dict[dev_eui] = 0

      # Split the data into lines
      lines = result_file_content.split('\n')

      for line in lines:

        # Controlla se la riga è esattamente nella forma "dev_eui: X"
        match = re.fullmatch(r"dev_eui:\s*\d+", line.strip())

        # Se la riga NON è esattamente nella forma "dev_eui: X", passa alla prossima iterazione
        if not match:
            continue

        # Trova tutti i numeri interi dopo "dev_eui:"
        dev_euis = re.findall(r"dev_eui:\s*(\d+)", line)
        #print("ECCO I DEV:",dev_euis,"\n\n")

        # Creare il dizionario con chiave e valore uguali
        results_dict[dev_euis[0]] = dev_euis[0]

      print("Target dict: ",target_dict)
      print("Results dict: ",results_dict)

      # Let's count the true positive.
      true_positive = 0
      for key in results_dict:
        if key in target_dict:
          if results_dict[key] == target_dict[key]:
            true_positive += 1
      print("True positive so far: ",true_positive)


      # Let's count the false positive.
      false_positive = 0
      for key in results_dict:
        if key not in target_dict:
          false_positive += 1
        else:
          if key in target_dict and results_dict[key] != target_dict[key]:
            false_positive += 1
      print("False positive so far: ",false_positive)

      # Let's count the false negative.
      false_negative = 0
      for key in target_dict:
        if key not in results_dict:
          false_negative += 1
      print("False negative so far: ",false_negative)

      total_true_positive += true_positive
      total_false_positive += false_positive
      total_false_negative += false_negative


  print("The precision is: ",total_true_positive/(total_true_positive+total_false_positive))
  print("The recall is: ",total_true_positive/(total_true_positive+total_false_negative))

##Let's evaluate Codestral

In [28]:
dataset_path = "prompt_Q2/"
result_path = "codestral_results/"
calculate_precision_recall(dataset_path,result_path)



 * * * Sample:  prompt_n_hours=2_skip_hours=836.txt
dev_eui: 120 
 dev_eui: 124 
 dev_eui: 128


dev_eui: 120
dev_eui: 124
dev_eui: 128
Target dict:  {'120': '120', '124': '124', '128': '128'}
Results dict:  {'120': '120', '124': '124', '128': '128'}
True positive so far:  3
False positive so far:  0
False negative so far:  0


 * * * Sample:  prompt_n_hours=2_skip_hours=198.txt
dev_eui: 121


dev_eui: 121
dev_eui: 123
dev_eui: 127
dev_eui: 120
dev_eui: 128
Target dict:  {'121': '121'}
Results dict:  {'121': '121', '123': '123', '127': '127', '120': '120', '128': '128'}
True positive so far:  1
False positive so far:  4
False negative so far:  0


 * * * Sample:  prompt_n_hours=2_skip_hours=286.txt
dev_eui: 123 
 dev_eui: 126 
 dev_eui: 128


dev_eui: 123
dev_eui: 128
Target dict:  {'123': '123', '126': '126', '128': '128'}
Results dict:  {'123': '123', '128': '128'}
True positive so far:  2
False positive so far:  0
False negative so far:  1


 * * * Sample:  prompt_n_hours=2_skip_h

## Let's evaluate Codestral with CoC (Chain of Code)

In [29]:
dataset_path = "prompt_Q2/"
result_path = "codestral_source_code_results/"
calculate_precision_recall(dataset_path,result_path)



 * * * Sample:  prompt_n_hours=2_skip_hours=836.txt
dev_eui: 120 
 dev_eui: 124 
 dev_eui: 128


dev_eui: 126
dev_eui: 125
dev_eui: 128
dev_eui: 124
dev_eui: 121
dev_eui: 122
dev_eui: 123
dev_eui: 127
dev_eui: 129
dev_eui: 120

Target dict:  {'120': '120', '124': '124', '128': '128'}
Results dict:  {'126': '126', '125': '125', '128': '128', '124': '124', '121': '121', '122': '122', '123': '123', '127': '127', '129': '129', '120': '120'}
True positive so far:  3
False positive so far:  7
False negative so far:  0


 * * * Sample:  prompt_n_hours=2_skip_hours=198.txt
dev_eui: 121


dev_eui: 126
dev_eui: 128
dev_eui: 121
dev_eui: 124
dev_eui: 122
dev_eui: 123
dev_eui: 127
dev_eui: 120

Target dict:  {'121': '121'}
Results dict:  {'126': '126', '128': '128', '121': '121', '124': '124', '122': '122', '123': '123', '127': '127', '120': '120'}
True positive so far:  1
False positive so far:  7
False negative so far:  0


 * * * Sample:  prompt_n_hours=2_skip_hours=286.txt
dev_eui: 123 
 dev

# Zip and download the results using the two approaches.

In [30]:
!zip -r codestral_results_q2.zip codestral_results
!zip -r codestral_source_code_results_q2.zip codestral_source_code_results

  adding: codestral_results/ (stored 0%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=356.txt (deflated 63%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=604.txt (deflated 73%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=540.txt (deflated 45%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=816.txt (deflated 79%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=322.txt (deflated 61%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=332.txt (deflated 45%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=970.txt (deflated 76%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=448.txt (deflated 28%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=984.txt (deflated 53%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=28.txt (deflated 53%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=912.txt (deflated 53%)
  adding: codestral_results/prompt_n_hours=2_skip_hours=242.txt (deflated 63%)
  adding: co